- Notebook réalisé par Aristide LALOUX, Hugo QUENIAT et Mohamed Ali SRIR.

# Bibliographie

- Deep Reinforcement Learning with Double Q-learning, Hado V. Hasselt et al, NIPS 2015: https://arxiv.org/abs/1509.06461
- Train a Mario-playing RL Agent, Yuansong Feng, Suraj Subramanian, Howard Wang et Steven Guo,  GitHub 2020 : https://github.com/pytorch/tutorials/blob/master/intermediate_source/mario_rl_tutorial.py
- Super Mario Bros for OpenAI Gym, Christian Kauten, GitHub 2018 : https://github.com/Kautenja/gym-super-mario-bros


# Mise en place de l'environnement de jeu

## Chargement de l'environnement du jeu, de l'émulateur

	Notre simulation va avoir lieu sur SuperMarioBros donc nous chargeons le jeu adapté. En outre, OpenAI Gym dispose de nombreux environnements de jeux anciens (Atari, NES etc) que vous pouvez utiliser avec une IA entrainée de façon analogue.

	Initialement, le jeu permet, en comptabilisant toutes les combinaisons d'actions possibles sur un controller NES, d'effectuer 256 actions distinctes. Cependant, dans un tel cas la table Q atteint des dimmensions bien trop élevées. Dès lors, on utilise la fonction JoypadSpace qui permet de simplifier les commandes possibles au sein du jeu.  L'ensemble des actions permises est ainsi réduit à 7 actions, les suivantes :

['NOOP'] (aucun mouvement)

['right'] (l'agent se déplace sur la droite)

['right', 'A'] (l'agent se déplace sur la droite et saute)

['right', 'B']  (l'agent se déplace sur la droite et envoie un projectile)

['right', 'A', 'B'] (l'agent se déplace sur la droite, saute et envoie un projectile)

['A'] (l'agent saute)

['left'] (l'agent se déplace sur la gauche)

In [ ]:
# Installation des dépendances et de Mario

!{sys.executable} -m pip install gym
!{sys.executable} -m pip install gym_super_mario_bros==7.3.0 nes_py
!{sys.executable} -m pip install torch torchvision torchaudio
!{sys.executable} -m pip intall pathlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib

In [1]:
# Importation des dépendances et du jeu

import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import random
import math
import gym

In [ ]:
#Génération du jeu dans sa configuration avec 7 actions accessibles
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Test du bon fonctionnement du jeu


L'utilisateur a deux options pour exécuter ce test:
 - laisser l'agent choisir de façon complètement aléatoire chaque action (pour chaque état la politique de décision est uniforme)
 - déterminer sa propre politique de décision qui sera appliquée pour chaque état

In [1]:
# Paramètres modifiables par l'utilisateur pour le test
No_Images = 10000 #Nombre d'images à parcourir avant que la simulation ne prenne fin.
Random_Actions = True #Indiquer si on souhaite voir l'IA prendre des actions complètement aléatoires
Actions_Law= [0, 1, 0, 0, 0, 0, 0] #loi de probabilité sur l'action à prendre à tout 

In [ ]:
# Lancement du jeu en fonction des paramètres choisis ci-dessus

if not Random_Actions:
	law=[]
	for i in range(7):
		law+=[i]*math.floor(100*Actions_Law[i])
done = True
# Jouer un nombre 'No_Images' d'images du jeu (frames)
for step in range(No_Images):
	if done :
		env.reset()
	if Random_Actions :
		action = env.action_space.sample()
	else : 
		action = law[random.randint(0,99)]
	state, reward, done, info = env.step(action)
	env.render()
env.close()

# Formation de l'agent via Double DQN

In [ ]:
# Import des dépendances pour prétraiter les images du jeu

from gym.wrappers import FrameStack, GrayScaleObservation
from gym.spaces import Box
import torch
from torch import nn
from torchvision import transforms as T
from collections import deque
import datetime, os, copy
import numpy as np
from pathlib import Path
import torch.optim as optim



## Couches de prétraitement
- Prise en compte d'une seule image par paquet reçu.
- Traitement des images prises en compte en groupe.
- Décolorisation des images, passage en niveau de gris (Réduction par 3 de la dimension, RGB -> Gris)
- Sous-échantillonnage des images pour conserver une image de taille inférieure.

In [5]:
# Paramètres de prétraitement modifiables par l'utilisateur
# Néanmoins, nous vous conseillons de ne pas trop toucher à ces paramètres (surtout à la taille des images)
pas_Images = 4 # Une action concerne pas_Images (autrement dit, un nombre pas_Images est considéré comme une seule image)
paquet_Images = 4 # Le traitement dans le réseau de neurones verra en entrée un paquet de paquet_Images images. Cela permet d'inclure la notion de mouvement du personnage
taille = 84 # Paramètre de sous-échantillonnage de l'image, ne conserver que taille x taille pixels pour l'image

In [6]:
# Couches de prétraitement

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        # Afin d'accélérer le processus de convergence nous n'utilisons que 1 image/skip.
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        # Comme nous n'utilisons qu'une image toutes les skip images, durant ces skip images nous utilisons la même action
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulation des récompenses et répétition de la même actiion durant les skip images
            state, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return state, total_reward, done, info


class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        # Conversion du tableau [Hauteur, Largeur, Couleur] en un tenseur [Couleur, Hauteur, Largeur]
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
		# Conversion de l'image en niveaux de gris.
        observation = transform(observation)
        return observation


class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)
		#Dimension de l'espace voulu
        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape), T.Normalize(0, 255)]
        )
		#Normalisation des valeurs depuis [0,255] à [0,1] puis sosu échantillonnage à la taille demandée
        observation = transforms(observation).squeeze(0)
        return observation


# Application des couches de prétraitement à notre environnement
env = SkipFrame(env, skip=pas_Images)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=taille)
env = FrameStack(env, num_stack=paquet_Images) 

## Structuration de l'agent

In [7]:
# Définition des méthodes de l'agent

class Mario:
    def __init__():
        pass

    def act(self, state):
        #Choix de l'action en fonction de l'état state de l'agent et la tactique epsilon-greedy
        pass

    def cache(self, experience):
        #Permet d'ajouter à sa mémoire l'expérience
        pass

    def recall(self):
        #Faire appel aux expériences précédentes via la mémoire de l'agent
        pass

    def learn(self):
        #Copie à partir des expériences dans la table Qtarget
        pass

### Actions

À chaque état, en fonction du ${\epsilon}$ de la stratégie ${\epsilon}$-greedy, l'agent peut choisir d'effectuer l'action la plus optimale selon sa table $Q$ (on dit qu'il exploite) ou une action totalement aléatoire (on dit qu'il explore).

In [8]:
# Paramètres modifiables par l'utilisateur pour l'entrainement de l'agent
exploration_rate_initial = 1 #Le epsilon de la stratégie epsilon-greedy. Il est en général mis à une valeur très élevée (très proche de 1 ou égale à 1) au début de l'entrainement de l'entrainement afin que l'agent puisse explorer toutes les actions et les états.
exploration_rate_coefficient_multiplicateur = 0.99999975 #Raison de la suite géométrique définissant la décroissance de l'exploration rate
exploration_rate_min = 0.1 #Exploration_rate minimal, au minimum l'agent fera une action aléatoire avec probabilité exploration_rate_min
Evaluation_Mode = False #Souhaite-t-on évaluer l'agent ?
Periodic_Evaluation = True #Souhaite-t-on périodiquement évaluer l'agent ?
Evaluation_every_step = 10 #Pas d'évaluation
pas_enregistrement = 5 #A quelle fréquence le log doit-il enregistrer les valeurs observées durant un épisode
sauvegarde_Mémoire = 1e5 # Nombre d'actions nécessaires de l'agent avant qu'il exécute une sauvegarde du réseau de neurones en mémoire.


In [9]:
# Gestion du choix et du traitement de l'action à effectuer

class Mario:
    def __init__(self, state_dim, action_dim, save_dir):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir

        self.use_cuda = torch.cuda.is_available()

        # Le réseau de neurones profond qui permet à Mario de prédire l'action optimale à réaliser
        self.net = MarioNet(self.state_dim, self.action_dim).float()
        if self.use_cuda:
            self.net = self.net.to(device="cuda")

        self.exploration_rate = exploration_rate_initial
        self.exploration_rate_decay = exploration_rate_coefficient_multiplicateur
        self.exploration_rate_min = exploration_rate_min
        self.curr_step = 0

        self.save_every = sauvegarde_Mémoire

    def act(self, state):
		#Choix de l'action suivant la stratégie epsilon-greedy
        # EXPLORATION
        if np.random.rand() < self.exploration_rate and (not Evaluation_Mode):
            action_idx = np.random.randint(self.action_dim)

        # EXPLOITATION
        else:
            state = state.__array__()
            if self.use_cuda:
                state = torch.tensor(state).cuda()
            else:
                state = torch.tensor(state)
            state = state.unsqueeze(0)
            action_values = self.net(state, model="online")
            action_idx = torch.argmax(action_values, axis=1).item()

        # Cécroissaince de l'exploration_rate jusqu'au minimum choisi.
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # Incrémentation du nombre d'étapes
        self.curr_step += 1
        return action_idx

## Entrainement de l'agent

### Fondement de l'algorithme de Double DQN

L'algorithme de Double DQN s'appuie sur le Double Q-learning et le DQN afin de former une méthode d'apprentissage profond par renforcement qui limite la surestimation de l'action value fonction.

Premièrement, l'héritage des Deep Q Networks (DQN) vient de l'utilisation d'un réseau dit "target" (cible), d'un réseau dit "online" (en ligne) et de l'experience replay. Ainsi, l'experience replay est utilisé afin de mettre à jour le réseau online. Le réseau target est ensuite périodiquement copié sur le réseau online. Néanmoins, chaque approximation dans l'équation de Bellman se fait en utilisant le réseau target, les mises à jour courantes ne sont ainsi rééutilisées que plus tard.

Deuxièmement, l'héritage du Double Q-learning provient de l'apprentissage de deux tables Q distinctes, associées à deux séries de poids : les poids du réseau online et les poids du réseau target. Les poids du réseau online servent à choisir la greedy policy tandis que les poids du réseau target permettent de l'évaluer.

Finalement, on utilise la table/ le réseau (ils sont associés) online pour évaluer la greedy policy à utiliser tandis qu'on utilise la table/ le réseau target pour en déterminer la valeur.


### Experience replay

Afin de mettre à jour les tables $Q_{target}$ et $Q_{online}$, l'agent fera appel (voir cellules suivantes) à l'experience replay : il apprend d'un certain échantillon de ses expériences précédentes pour déterminer les meilleures stratégies à adopter. Nous mettons ainsi en place un processus de sauvegarde en mémoire et d'accès à cette même mémoire.

In [10]:
# Gestion de la mémoire

class Mario(Mario):  #sous-classe pour hériter des méthodes et attributs pré-annoncées et déjà écrits
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.memory = deque(maxlen=100000) #la mémoire de l'agent
        self.batch_size = 32 #nombre d'expériences à utiliser pour mettre à jour la table Qtarget à chaque accès à la mémoire

    def cache(self, state, next_state, action, reward, done):
		#L'entrée est en LazyFrame pour state et next_state, format utilisé par Gym. Il est nécessaire de les convertir en array Numpy pout utiliser torch.
        state = state.__array__()
        next_state = next_state.__array__()

		#Pytorch a deux options, utiliser le GPU (Cuda) ou le CPU (pas Cuda)
        if self.use_cuda:
            state = torch.tensor(state).cuda()
            next_state = torch.tensor(next_state).cuda()
            action = torch.tensor([action]).cuda()
            reward = torch.tensor([reward]).cuda()
            done = torch.tensor([done]).cuda()
        else:
            state = torch.tensor(state)
            next_state = torch.tensor(next_state)
            action = torch.tensor([action])
            reward = torch.tensor([reward])
            done = torch.tensor([done])
		#Stockage en mémoire (un buffer en fait) des expériences de l'agent
        self.memory.append((state, next_state, action, reward, done,))

    def recall(self):
		#On recueille un échantillon aléatoire des expériences stockées en mémoire pour mettre à jour la table Qtarget
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(torch.stack, zip(*batch))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

### CNN - Réseau de neurones convolutionnel

Le réseau utilisé ici consiste en trois convolutions suivies d'un Relu, puis d'un flatten et enfin de deux couches fully connected linéaires.

In [ ]:
# Description du réseau de neurones convolutionnel

class MarioNet(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w = input_dim
		#s'assurer que les dimensions de l'image en entrée du CNN sont bien conformes à ce qui est attendu post-prétraitement.
        if h != 84:
            raise ValueError(f"Expecting input height: 84, got: {h}")
        if w != 84:
            raise ValueError(f"Expecting input width: 84, got: {w}")
		#Le CNN en lui-même
        self.online = nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

        self.target = copy.deepcopy(self.online)

        # Les paramètres de la table Q_target restent inchangés, on indique donc de ne pas l'impliquer dans la descente de gradient.
		# Elle sera synchronisée avec Q_online périodiquement.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

In [ ]:
# Sauvegarde du réseau de neurones pour utilisation ultérieure

class Mario(Mario):
    def save(self):
        save_path = (
            self.save_dir / f"mario_net_{int(self.curr_step)}.chkpt"
        )
        torch.save(
            dict(model=self.net.state_dict(), exploration_rate=self.exploration_rate),
            save_path,
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")

### Application de la méthode du Double DQN

- TD estimate : estimation de la greedy policy pour un état s
#
- - ${TD}_e = Q_{online}^*(s,a)$
#
#
- TD Target : détermination de la valeur de la greedy policy à partir de l'état s' qui suit et r le reward courant
#
- - $a' = argmax_{a} Q_{online}(s', a)$
- - ${TD}_t = r + \gamma Q_{target}^*(s',a')$

In [28]:
# Calcul de TD estimate et TD target

class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
		# Discount factor, permet d'assurer la convergence de la fonction de gain J(pi)
        self.gamma = 0.9

    def td_estimate(self, state, action):
        current_Q = self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ]  # TD_estimate = Q_online(s,a); on choisit la valeur maximale de la table Q online pour l'état s dans lequel on se trouve.
        return current_Q
	# Ne pas faire de descente de gradient sur le réseau target
    @torch.no_grad()
    def td_target(self, reward, next_state, done):
		# On détermine l'état suivant
        next_state_Q = self.net(next_state, model="online")
		# On détermine l'action à prendre dans l'état suivant
        best_action = torch.argmax(next_state_Q, axis=1)
		# On détermine la valeur de Q_target associée
        next_Q = self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float() # TD_target = current_reward + gamma *  Q_target(s',a')

### Mise à jour du modèle

On applique l'algorithme de descente de gradient pour mettre à jour les poids du réseau online (qui seront ensuite périodiquement copiés sur le réseau target). On note $\alpha$ le learning rate, dans ]0;1] il permet de limiter ou non l'impact du bruit.

- - $\theta_{online} \leftarrow \theta_{online} + \alpha \nabla(TD_e - TD_t)$

In [29]:
# Mise à jour du modèle

class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
		#Algorithme de descente de gradient avec un learning rate lr : algortihme d'Adam
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
		#Mesure de différence dans une série statistique, écart L^1 quand l'écart est supérieur à 1 et écart quadratique sinon
        self.loss_fn = torch.nn.SmoothL1Loss()

    def update_Q_online(self, td_estimate, td_target):
		#On mesure la différence entre les tables online et target 
        loss = self.loss_fn(td_estimate, td_target)
		#On vide le gradient avant de le calculer à nouveau
        self.optimizer.zero_grad()
		#Calcul du gradient
        loss.backward()
		#Les paramètres du réseau de neurones sont mises à jour à partir de ce gradient
        self.optimizer.step()
		#Renvoi de la différence, qui sera ensuite exploitée pour mesurer l'efficacité de l'épisode
        return loss.item()

    def sync_Q_target(self):
		#Synchronisation de la table Q_target
        self.net.target.load_state_dict(self.net.online.state_dict())

In [ ]:
# Phase d'apprentissage et de reports des expériences sur les modèles

class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.burnin = 1e4  # nombre d'expériences avant le début de l'entrainement
        self.learn_every = 3  # chaque mise à jour de Q_online se fait tous les learn_every
        self.sync_every = 1e4  # période de synchronisation de Q_target

    def learn(self):
        if self.curr_step % self.sync_every == 0:
            self.sync_Q_target()
        if self.curr_step % self.save_every == 0:
            self.save()
        if self.curr_step < self.burnin:
            return None, None
        if self.curr_step % self.learn_every != 0:
            return None, None
        #Extraction d'une expérience depuis la mémoire
        state, next_state, action, reward, done = self.recall()
        td_est = self.td_estimate(state, action)
        td_tgt = self.td_target(reward, next_state, done)
		#Mise à jour de la table Q_online
        loss = self.update_Q_online(td_est, td_tgt) # on utilise
		#Renvoi de la moyenne de la table Q sur cet épisode et de la différence avec la table target
        return (td_est.mean().item(), loss)

# Lancement du jeu pour évaluer l'IA ou l'entrainer

In [32]:
# Sauvegarde de données de l'IA pour étude, posttraitement humain et réutilisation ultérieure.

import matplotlib.pyplot as plt
import time

class MetricLogger:
    def __init__(self, save_dir):
		#Rapport d'un épisode à transcrire sur le fichier log
        self.save_log = save_dir / "log"
        with open(self.save_log, "w") as f:
            f.write(
                f"{'Episode':>8}{'Step':>8}{'Epsilon':>10}{'MeanReward':>15}"
                f"{'MeanLength':>15}{'MeanLoss':>15}{'MeanQValue':>15}"
                f"{'TimeDelta':>15}{'Time':>20}\n"
            )
		#Diffèrents graphiques
        self.ep_rewards_plot = save_dir / "reward_plot.jpg"
        self.ep_lengths_plot = save_dir / "length_plot.jpg"
        self.ep_avg_losses_plot = save_dir / "loss_plot.jpg"
        self.ep_avg_qs_plot = save_dir / "q_plot.jpg"

        # Évolution des données qui permettent d'évaluer l'agent et son entrainemen t
        self.ep_rewards = []
        self.ep_lengths = []
        self.ep_avg_losses = []
        self.ep_avg_qs = []

        # Moyennes glissantes
        self.moving_avg_ep_rewards = []
        self.moving_avg_ep_lengths = []
        self.moving_avg_ep_avg_losses = []
        self.moving_avg_ep_avg_qs = []

        # Début de l'enregistrement d'un épisode
        self.init_episode()

        # Heure de l'enregistrement
        self.record_time = time.time()

	#Ajout à l'enregistrement des valeurs observées par l'action effectuée
    def log_step(self, reward, loss, q):
        self.curr_ep_reward += reward
        self.curr_ep_length += 1
        if loss:
            self.curr_ep_loss += loss
            self.curr_ep_q += q
            self.curr_ep_loss_length += 1
	#Après la fin de l'épisode, ajoutde toutes les valeurs ajoutées aux différentes historiques)
    def log_episode(self):
        self.ep_rewards.append(self.curr_ep_reward)
        self.ep_lengths.append(self.curr_ep_length)
        if self.curr_ep_loss_length == 0:
            ep_avg_loss = 0
            ep_avg_q = 0
        else:
            ep_avg_loss = np.round(self.curr_ep_loss / self.curr_ep_loss_length, 5)
            ep_avg_q = np.round(self.curr_ep_q / self.curr_ep_loss_length, 5)
        self.ep_avg_losses.append(ep_avg_loss)
        self.ep_avg_qs.append(ep_avg_q)
		#fin des enregistrements, lancement d'un nouvel enregistrement pour l'épisode suivant
        self.init_episode()

    def init_episode(self):
		#Au début de l'épisode tous les compteurs sont réinitialisés
        self.curr_ep_reward = 0.0
        self.curr_ep_length = 0
        self.curr_ep_loss = 0.0
        self.curr_ep_q = 0.0
        self.curr_ep_loss_length = 0

    def record(self, episode, epsilon, step):
		#Calcul des moyennes glissantes et ajouts
        mean_ep_reward = np.round(np.mean(self.ep_rewards[-100:]), 3)
        mean_ep_length = np.round(np.mean(self.ep_lengths[-100:]), 3)
        mean_ep_loss = np.round(np.mean(self.ep_avg_losses[-100:]), 3)
        mean_ep_q = np.round(np.mean(self.ep_avg_qs[-100:]), 3)
        self.moving_avg_ep_rewards.append(mean_ep_reward)
        self.moving_avg_ep_lengths.append(mean_ep_length)
        self.moving_avg_ep_avg_losses.append(mean_ep_loss)
        self.moving_avg_ep_avg_qs.append(mean_ep_q)
		
		#Détermination des temps de calcul entre deux instants d'enregistrement
        last_record_time = self.record_time
        self.record_time = time.time()
        time_since_last_record = np.round(self.record_time - last_record_time, 3)

		#Affichage dans la console et sur le log
        print(
            f"Episode {episode} - "
            f"Step {step} - "
            f"Epsilon {epsilon} - "
            f"Mean Reward {mean_ep_reward} - "
            f"Mean Length {mean_ep_length} - "
            f"Mean Loss {mean_ep_loss} - "
            f"Mean Q Value {mean_ep_q} - "
            f"Time Delta {time_since_last_record} - "
            f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        )

        with open(self.save_log, "a") as f:
            f.write(
                f"{episode:8d}{step:8d}{epsilon:10.3f}"
                f"{mean_ep_reward:15.3f}{mean_ep_length:15.3f}{mean_ep_loss:15.3f}{mean_ep_q:15.3f}"
                f"{time_since_last_record:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )
		#Mise à jour des graphiques
        for metric in ["ep_rewards", "ep_lengths", "ep_avg_losses", "ep_avg_qs"]:
            plt.plot(getattr(self, f"moving_avg_{metric}"))
            plt.savefig(getattr(self, f"{metric}_plot"))
            plt.clf()

In [ ]:
# Faire jouer l'IA au jeu

#Utilisation du GPU NVIDIA si possible pour accélérer le processus
use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
print()

#Création du dossier pour sauvegarder les graphiques, log et checkpoints
save_dir = Path("checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
save_dir.mkdir(parents=True)

#Notre agent
mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir)

#Classe permettant la sauvegarde des infos
logger = MetricLogger(save_dir)

#Mode : training ou evaluation
evaluation = Evaluation_Mode

#Nombre d'épisodes à entrainer l'agent
episodes = 40
for e in range(episodes):
	#Réinitialisation du monde pour nouvel essai
	state = env.reset()
	#Début du jeu
	#On décide d'afficher une tentative périodiquement pour voir l'évolution ou à chaque épisode si on ets dans un mode d'évaluation
	evaluation = (e%Evaluation_every_step== 0 and Periodic_Evaluation) or (Evaluation_Mode) 
	while True:
		if (mario.curr_step % mario.save_every == 0) :
			mario.save()
		if (not evaluation):
			# L'agent choisit l'action à effectuer en fonction de l'état dans lequel il est
			action = mario.act(state)
			# L'agent applique l'action dans le jeu
			next_state, reward, done, info = env.step(action)
			# L'agent enregistre l'expérience effectuée
			mario.cache(state, next_state, action, reward, done)
			# L'agent apprend 
			q, loss = mario.learn()
			# Enregistrement de l'évolution
			logger.log_step(reward, loss, q)
			# Passage au nouvel état
			state = next_state
		else:
			# L'agent choisit l'action à effectuer en fonction de l'état dans lequel il est
			action = mario.act(state)
			# L'agent applique l'action dans le jeu
			next_state, reward, done, info = env.step(action)
			#On affiche l'évaluation
			env.render()
			# L'agent enregistre l'expérience effectuée
			mario.cache(state, next_state, action, reward, done)
			# Enregistrement de l'évolution
			logger.log_step(reward, loss, q)
			# Passage au nouvel état
			state = next_state
		# Vérifier si le jeu est fini (réussite ou mort)
		if done or info["flag_get"]:
			if evaluation:
				env.close()
			break
	# Enregistrement de l'épisode
	logger.log_episode()
	# Affichage de l'enregistrement tous les pas_enregistrement
	if e % pas_enregistrement == 0:
		logger.record(episode=e, epsilon=mario.exploration_rate, step=mario.curr_step)